## Ejecicio 4 DS
Definición de clases:

In [3]:
from abc import ABC, abstractmethod
import math
from enum import Enum

class EstadoMotor(Enum):
    ACELERANDO = 1
    FRENANDO = 2
    APAGADO = 3
    ENCENDIDO = 4

class Filtro(ABC):
    @abstractmethod
    def ejecutar(self, revoluciones, estado_motor):
        pass

class CadenaFiltros:
    def __init__(self):
        self.filtros = []
        self.objetivo = None

    def addFiltro(self, filtro):
        self.filtros.append(filtro)

    def ejecutarCadena(self, revoluciones, estado_motor):
        for filtro in self.filtros:
            revoluciones = filtro.ejecutar(revoluciones, estado_motor)
        return revoluciones

    def setTarget(self, objetivo):
        self.objetivo = objetivo
        
class GestorFiltros:
    def __init__(self, objetivo):
        self.cadenaFiltros = CadenaFiltros()
        self.cadenaFiltros.setTarget(objetivo)

    def addFiltro(self, filtro):
        self.cadenaFiltros.addFiltro(filtro)

    def peticionFiltros(self, revoluciones, estado_motor):
        self.cadenaFiltros.ejecutarCadena(revoluciones, estado_motor)
        

class RepercutirRozamiento(Filtro):
    def __init__(self):
        self.rozamiento = 0
    
    def ejecutar(self, revoluciones, estado_motor):
        # Suponiendo que el rozamiento disminuye constantemente en 1
        self.rozamiento -= 1
        # Aquí podrías realizar cualquier otro cálculo necesario antes de pasar las revoluciones al siguiente filtro
        revoluciones += self.rozamiento  # Ajustamos las revoluciones considerando el rozamiento
        return revoluciones

class CalcularVelocidad(Filtro):
    def ejecutar(self, revoluciones, estado_motor):
        radio_eje = 0.15  # metros
        v = 2 * math.pi * radio_eje * revoluciones * (60 / 1000)  # Calculamos la velocidad lineal en km/h
        return v
    
    
class Objetivo:
    def __init__(self):
        self.revoluciones = 0
        self.velocidad = 0
        self.distancia = 0

    def ejecutar(self, revoluciones, estado_motor):
        # Actualizamos las revoluciones teniendo en cuenta el estado del motor y el rozamiento
        # Suponemos que el rozamiento disminuye constantemente en 1
        rozamiento = 1 if estado_motor == EstadoMotor.ACELERANDO else 0
        self.revoluciones = revoluciones - rozamiento
        
        # Calculamos la velocidad lineal en km/h
        radio_eje = 0.15  # metros
        self.velocidad = 2 * math.pi * radio_eje * self.revoluciones * (60 / 1000)
        
        # Actualizamos la distancia recorrida
        # Suponiendo un intervalo de tiempo fijo de 1 segundo por simplicidad
        self.distancia += self.velocidad / 3600  # Convertimos la velocidad de km/h a m/s

        # Devolvemos las revoluciones actualizadas
        return self.revoluciones
